In [1]:
import urllib.request
import urllib
import re
import pandas as pd
import math

In [2]:
CVDfile = './cvd.csv'
df = pd.read_csv(CVDfile)

In [3]:
df.head()
meshDes = []
mesh = []
for i in df['name']:
    #print(i)
    name = i.split()[0]
    name = name.split(',')[0]
    meshDes.append(name)
for i in meshDes:
    if i not in mesh:
        mesh.append(i)
print(len(mesh))

255


In [4]:
urls = []
for i in mesh:
    url = "https://reactome.org/content/query?q=" + i + "&species=Homo+sapiens&types=Pathway&cluster=true"
    #print(url)
    urls.append(url)

In [5]:
dic = {}
l = ['1', '2']
dic[mesh[0]] = l
dic[mesh[1]] = l
dic


{'Cardiovascular': ['1', '2'], 'Heart': ['1', '2']}

In [6]:
len(urls)

255

## Find the whole number of the results

In [7]:
dic = {}
for i in range(len(urls)):
    url = urls[i]
    flag = 0
    try:
        request = urllib.request.Request(url)
        response = urllib.request.urlopen(request)
        content = response.read().decode('ISO-8859-1')
        #print(content)
        pattern = re.compile('Filter.*?<strong>(.*?)</strong>.*of.*<strong>(.*?)</strong>')
        pages = re.findall(pattern, content)
        #print(pages)
        print(mesh[i],' is successful!')
        
        x = int(pages[0][0])
        n = int(pages[0][1])
        pageSum = math.ceil(n/x)
        
        temp = []

        for j in range(pageSum):
            url_page = url + '&page=' + str(j+1)
            try:
                request = urllib.request.Request(url_page)
                response = urllib.request.urlopen(request)
                content = response.read().decode('ISO-8859-1')
                #print(content)
                pattern = re.compile('<strong>Identifier: </strong>(R-HSA-\d+)')
                items = re.findall(pattern, content)
                for k in items:
                    temp.append(k)

            except urllib.request.URLError as e:
                if hasattr(e,"code"):
                    print(e.code)
                if hasattr(e,"reason"):
                    print(e.reason)
                    
        dic[mesh[i]] = temp

    except urllib.request.URLError as e:
        print(mesh[i], 'is failed!')
        #if hasattr(e,"code"):
        #    print(e.code)
        #if hasattr(e,"reason"):
        #    print(e.reason)

Cardiovascular  is successful!
Heart  is successful!
22q11  is successful!
DiGeorge  is successful!
Alagille  is successful!
Aortic  is successful!
Arrhythmogenic is failed!
404
Not Found
Barth  is successful!
Cor  is successful!
Coronary  is successful!
Anomalous  is successful!
Bland  is successful!
Myocardial  is successful!
Crisscross is failed!
404
Not Found
Dextrocardia is failed!
404
Not Found
Kartagener is failed!
404
Not Found
Ductus is failed!
404
Not Found
Ebstein  is successful!
Ectopia  is successful!
Eisenmenger is failed!
404
Not Found
Aortopulmonary is failed!
404
Not Found
Truncus is failed!
404
Not Found
Endocardial is failed!
404
Not Found
Foramen is failed!
404
Not Found
Lutembacher is failed!
404
Not Found
Heterotaxy  is successful!
Hypoplastic  is successful!
Isolated  is successful!
LEOPARD  is successful!
Levocardia is failed!
404
Not Found
Marfan  is successful!
Noonan  is successful!
Tetralogy is failed!
404
Not Found
Transposition  is successful!
Congenitally

In [18]:
name = []
Indent = []

for i in dic:
    for j in dic[i]:
        name.append(i)
        Indent.append(j)

#for i in range(len(name)):
    #print(name[i], ': ', Indent[i])
    
new = {'name':name, 'Identifier':Indent}
new = pd.DataFrame(new)

In [19]:
new.head()

,name,Identifier
0,Cardiovascular,R-HSA-9008059
1,Cardiovascular,R-HSA-392451
2,Cardiovascular,R-HSA-216083
3,Cardiovascular,R-HSA-211935
4,Cardiovascular,R-HSA-2022377


In [20]:
new.to_csv('./pathwayIndentifier.csv')